In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score

In [5]:
df = pd.read_csv('data-logistic.csv', header=None)
df.head()

,0,1,2
0,-1,-0.663827,-0.138526
1,1,1.994596,2.468025
2,-1,-1.247395,0.749425
3,1,2.309374,1.899836
4,1,0.849143,2.407750


In [60]:
X = df.loc[:,1:]
y = df.loc[:,0]


In [112]:
def w_calc(X, y, w, k, C=0):
    grad1, grad2 = 0, 0
    for i in range(0, len(y)):
        grad1 += y[i] * X[1][i] * (1.0 - 1.0 / (1.0 + np.exp(-y[i] * (w[0]*X[1][i] + w[1]*X[2][i]))))
        grad2 += y[i] * X[2][i] * (1.0 - 1.0 / (1.0 + np.exp(-y[i] * (w[0]*X[1][i] + w[1]*X[2][i]))))
    w[0] = w[0] + (k * (1.0 / len(y)) * grad1) 
    w[1] = w[1] + (k * (1.0 / len(y)) * grad2)
    return w   

In [64]:
def w_calc_l2(X, y, w, k, C):
    grad1, grad2 = 0, 0
    for i in range(0, len(y)):
        grad1 += y[i] * X[1][i] * (1.0 - 1.0 / (1.0 + np.exp(-y[i] * (w[0]*X[1][i] + w[1]*X[2][i]))))
        grad2 += y[i] * X[2][i] * (1.0 - 1.0 / (1.0 + np.exp(-y[i] * (w[0]*X[1][i] + w[1]*X[2][i]))))
    w[0] = w[0] + (k * (1.0 / len(y)) * grad1) - k * C * w[0]
    w[1] = w[1] + (k * (1.0 / len(y)) * grad2) - k * C * w[1]
    return w    

In [124]:
def GD_l2(X, y, C=10, k=0.1, w=[0,0], eps=1e-5, max_iter=10000):
    w_prev = [0, 0]
    for i in range(max_iter):
        w_prev[0], w_prev[1] = w[0], w[1]
        w = w_calc_l2(X, y, w, k, C)
        if np.sqrt((w_prev[0] - w[0]) ** 2 + (w_prev[1] - w[1]) ** 2) <= eps:
            break

    return w

In [100]:
def GD(X, y, C=0, k=0.1, w=[0,0], eps=1e-5, max_iter=10000):
    w_prev = [0, 0]
    for i in range(max_iter):
        w_prev[0], w_prev[1] = w[0], w[1]
        w = w_calc(X, y, w, k)
        if np.sqrt((w_prev[0] - w[0]) ** 2 + (w_prev[1] - w[1]) ** 2) <= eps:
            break

    return w

In [126]:
w = GD(X, y)
w_l2 = GD_l2(X, y)

In [127]:
print(w)
print(w_l2)

[0.2878513597515838, 0.09194655615666857]
[0.028559612385813844, 0.024781077163038526]


In [128]:
def a(X, w):
    return 1.0 / (1.0 + np.exp(-w[0] * X[1] - w[1] * X[2]))

y_pred = a(X, w)
y_pred_l2 = a(X, w_l2)

auc = roc_auc_score(y, y_pred)
auc_l2 = roc_auc_score(y, y_pred_l2)

print(round(auc,3),round(auc_l2,3))

0.927 0.936
